In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import pearsonr, permutation_test
import sys
import argparse
import h5py
from scipy.stats import t as student_t
from statsmodels.stats import multitest as mt

In [13]:
# IDP_df = pd.read_pickle('male_IDP.pkl')
vars_df = pd.read_pickle('female_vars.pkl')
# IDP_names = np.loadtxt("IDP_names.txt", dtype=str, delimiter='\n')
# IDP_categories = np.loadtxt("IDP_categories.txt", dtype=str, delimiter='\n')
with open('../Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
# vars_categories = np.loadtxt("../Analsysis Female New/vars_categories.txt", dtype=str, delimiter='\n')
# vars_categories = np.loadtxt("../Analsysis Female New/vars_categories.txt", dtype=str)

with open('../Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

In [14]:
vars_df.head()

,modality,bonf,fdr,dataframe
0,T1L_large_HGL,5.441192,3.425256,idx ...
1,T1L_large_SFCN,5.441192,3.113612,idx ...
2,T1L_large_SFCN_50b,5.441192,3.159399,idx ...
3,T1L_large_SFCN_50b10s,5.441192,3.236979,idx ...


In [15]:
df = vars_df.iloc[0].dataframe

In [16]:
df[df.log_p_values > 2]

,idx,names,Categories,pearson_r,t_test_statistic,p_values_corrected,p_values,abs_pearson_r,log_p_values,log_p_values_corrected
132,160,Father still alive (0.0),Lifestyle,-0.040368,2.832381,0.261474,0.004639,0.040368,2.333568,0.582572
149,182,Age first had sexual intercourse (0.0),Lifestyle,-0.050758,3.475740,0.059654,0.000514,0.050758,3.288972,1.224357
151,184,Age first had sexual intercourse (2.0),Lifestyle,-0.046285,3.072440,0.172505,0.002136,0.046285,2.670365,0.763199
152,186,Lifetime number of sexual partners (0.0),Lifestyle,0.054113,3.594708,0.045337,0.000328,0.054113,3.483710,1.343547
154,188,Lifetime number of sexual partners (2.0),Lifestyle,0.046755,2.990476,0.201530,0.002802,0.046755,2.552522,0.695661
...,...,...,...,...,...,...,...,...,...,...
13233,16845,Operative procedures - OPCS4 (T313 - T31.3 Exc...,Medical History,0.040901,2.900029,0.234121,0.003747,0.040901,2.426261,0.630560
13311,16923,Operative procedures - OPCS4 (U051 - U05.1 Com...,Medical History,0.041120,2.915599,0.227962,0.003566,0.041120,2.447846,0.642137
13337,16949,Operative procedures - OPCS4 (U211 - U21.1 Mag...,Medical History,0.047614,3.377058,0.078115,0.000738,0.047614,3.131810,1.107264
13356,16968,Operative procedures - OPCS4 (V256 - V25.6 Pri...,Medical History,0.039548,2.803999,0.273305,0.005067,0.039548,2.295275,0.563353


In [88]:
len(df[df.log_p_values > 5.441192])

56

In [77]:
vars_df

,modality,bonf,fdr,dataframe
0,T1L_large_HGL,5.441192,3.425256,idx ...
1,T1L_large_SFCN,5.441192,3.113612,idx ...
2,T1L_large_SFCN_50b,5.441192,3.159399,idx ...
3,T1L_large_SFCN_50b10s,5.441192,3.236979,idx ...


In [84]:
np.power(10, 5.441192)

276179.8567397032

In [86]:
276200 / 5000 * 2 * 345 / 3600 * 4

42.35066666666666

In [95]:
276200 / 5000 * 2 * 1676 / 3600 / 24

2.1431074074074075

In [8]:
subjects_to_be_ignored = [21269692]
subject_sex = 'female'
prototype_flag = True

deconfound_flag = True
vars_cutoff_nan = 0.1

# Identify Correct File to Load

IDP_file = '../Analsysis Female New/female_test_IDPs.h5'
confounds_file = '../Analsysis Female New/female_test_conf12.h5'
data_file = 'female_test.pkl'

confounds = h5py.File(confounds_file, 'r')
confounds = confounds['conf12'][()]

IDP_h5 = h5py.File(IDP_file, 'r')
subjects_h5 = IDP_h5['subjects'][()]

# Load the relevant dataframe based on the required modality

df = pd.read_pickle(data_file)

IDP_h5 = h5py.File(IDP_file, 'r')
vars_i_deconf = IDP_h5['vars_i_deconf'][()]
subjects_h5 = IDP_h5['subjects'][()]

with open('../Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('../Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

varsIDX = np.arange(0, len(varsHeader), 1).astype(int)

vars_perc_not_nan = np.sum(np.isnan(vars_i_deconf)==False,axis=0) / vars_i_deconf.shape[0]
vars_idx_nan_cutoff = np.where(vars_perc_not_nan >= vars_cutoff_nan)[0]

varsHeader_subset = varsHeader[vars_idx_nan_cutoff]
vars_i_deconf_subset = vars_i_deconf[:, vars_idx_nan_cutoff]
vars_categories_subset = vars_categories[vars_idx_nan_cutoff]
varsIDX_subset = varsIDX[vars_idx_nan_cutoff]

del IDP_h5

vars_corrs = []
vars_mods = []
vars_bnf_thrs = []
vars_fdr_thrs = []

In [10]:
vars_i_deconf_subset.shape

(5021, 13809)

In [11]:
varsIDX_subset

array([    0,     1,     2, ..., 17516, 17525, 17526])

In [20]:
varsIDX_interest = df[df.log_p_values > 2].idx.values

In [21]:
varsIDX_interest

array([  160,   182,   184,   186,   188,   418,   430,   704,   754,
         805,  1204,  1209,  1454,  1517,  1552,  1656,  1657,  1659,
        1817,  1866,  1942,  2024,  2027,  2059,  2075,  2225,  2301,
        2394,  2421,  2613,  2729,  2778,  2780,  2796,  2798,  2812,
        2844,  2865,  2870,  2873,  2922,  3019,  3021,  3036,  3048,
        3050,  3052,  3060,  3152,  3164,  3166,  3167,  3183,  3185,
        3187,  3189,  3191,  3218,  3220,  3222,  3224,  3230,  3231,
        3233,  3235,  3237,  3243,  3245,  3248,  3250,  3252,  3254,
        3259,  3261,  3264,  3270,  3272,  3274,  3278,  3281,  3285,
        3291,  3295,  3305,  3314,  3316,  3318,  3320,  3322,  3324,
        3330,  3331,  3332,  3334,  3336,  3338,  3340,  3347,  3354,
        3375,  3377,  3379,  3380,  3382,  3384,  3386,  3388,  3390,
        3392,  3396,  3398,  3400,  3401,  3402,  3404,  3406,  3408,
        3410,  3412,  3414,  3416,  3418,  3420,  3422,  3424,  3426,
        3428,  3430,

In [76]:
varsIDX_interest.shape

(345,)

In [23]:
vars_i_deconf_subset.shape

(5021, 13809)

In [24]:
varsIDX_subset.shape

(13809,)

In [26]:
a = varsIDX_interest[0]
np.where(varsIDX_subset==a)

(array([132]),)

In [27]:
varsIDX_subset[np.where(varsIDX_subset==a)]

array([160])

In [31]:
vars_i_deconf_subset[np.where(varsIDX_subset==a)[0][0]]

array([-0.29583353,         nan,         nan, ...,         nan,
       -0.52333289, -0.50139622])

In [33]:
# varsHeader_subset = varsHeader[vars_idx_nan_cutoff]
# vars_i_deconf_subset = vars_i_deconf[:, vars_idx_nan_cutoff]
# vars_categories_subset = vars_categories[vars_idx_nan_cutoff]
# varsIDX_subset = varsIDX[vars_idx_nan_cutoff]

In [40]:
varsHeader_subset[np.where(varsIDX_subset==a)[0][0]]

'Father still alive (0.0)'

In [36]:
vars_i_deconf_subset[np.where(varsIDX_subset==a)[0][0]]

array([-0.29583353,         nan,         nan, ...,         nan,
       -0.52333289, -0.50139622])

In [37]:
vars_categories_subset[np.where(varsIDX_subset==a)[0][0]]

'Lifestyle'

In [38]:
varsIDX_subset[np.where(varsIDX_subset==a)[0][0]]

160

In [92]:
sm = 0

for idx in range(4):
    df = vars_df.iloc[idx].dataframe
    varsIDX_interest = df[df.log_p_values > 2].idx.values

    print(len(varsIDX_interest))
    sm += len(varsIDX_interest)
    
    varsHeader_subset2 = []
    vars_i_deconf_subset2 = []
    vars_categories_subset2 = []
    varsIDX_subset2 = []

    for item in varsIDX_interest:
        varsHeader_subset2.append(varsHeader_subset[np.where(varsIDX_subset==item)[0][0]])
        vars_i_deconf_subset2.append(vars_i_deconf_subset[:, np.where(varsIDX_subset==item)[0][0]])
        vars_categories_subset2.append(vars_categories_subset[np.where(varsIDX_subset==item)[0][0]])
        varsIDX_subset2.append(varsIDX_subset[np.where(varsIDX_subset==item)[0][0]])
    
    filename = 'logpOver2_' + str(idx) + '.npz'
    
    np.savez(filename, np.array(varsHeader_subset2), np.array(vars_i_deconf_subset2),
        np.array(vars_categories_subset2), np.array(varsIDX_subset2))
    
print(sm)

345
467
441
423
1676


In [57]:
(np.array(varsIDX_subset2) == varsIDX_interest).all()

True

In [46]:
np.where(varsIDX_subset==item)[0][0]

5050

In [51]:
varsIDX_subset.shape

(13809,)

In [60]:
np.savez('logpOver2.npz', np.array(varsHeader_subset2), np.array(vars_i_deconf_subset2),
        np.array(vars_categories_subset2), np.array(varsIDX_subset2))

In [61]:
npzfile = np.load('logpOver2.npz')

In [67]:
npzfile.files

['arr_0', 'arr_1', 'arr_2', 'arr_3']

In [68]:
npzfile['arr_0']

array(['Father still alive (0.0)',
       'Age first had sexual intercourse (0.0)',
       'Age first had sexual intercourse (2.0)',
       'Lifetime number of sexual partners (0.0)',
       'Lifetime number of sexual partners (2.0)',
       'Attendance/disability/mobility allowance (2.0)',
       'Leisure/social activities (2.1)',
       'Time spent outdoors in winter (2.0)', 'Nap during day (0.0)',
       'Types of transport used (excluding work) (1.0)',
       'Never eat eggs, dairy, wheat, sugar (0.0)',
       'Never eat eggs, dairy, wheat, sugar (2.0)',
       'Type of yogurt eaten (2.0)', 'Englyst dietary fibre (2.0)',
       'Magnesium (2.0)', 'Added milk to instant coffee (1.0)',
       'Added milk to instant coffee (2.0)',
       'Added milk to instant coffee (4.0)', 'Oat crunch intake (2.0)',
       'Type milk consumed (1.0)',
       'Number of crackers/crispbreads with butter/margarine (2.0)',
       'Dessert consumers (1.0)', 'Dessert consumers (4.0)',
       'Sponge puddin

In [69]:
npzfile['arr_1']

array([[-2.54908761e-01,  6.09266683e-03,  9.66582546e-01, ...,
         8.34674183e-01,  1.76414362e-01,  6.92589060e-01],
       [-7.06110535e-01,  9.68633004e-01,  1.06990120e+00, ...,
        -3.05937050e-01, -2.46170675e-02, -1.31356283e+00],
       [-6.12134001e-01,             nan,  1.30053321e+00, ...,
         4.13251721e-02,  1.70997085e-02, -1.23939860e+00],
       ...,
       [ 2.14533575e-02,  1.22199895e-02,  1.21987648e-02, ...,
        -4.75416062e-03,  7.02836494e-03,  5.30153084e-03],
       [ 2.21096759e-04, -1.40524247e-02,  6.54376287e-04, ...,
         3.05060612e-03, -1.68734389e-03,  5.82131542e-03],
       [ 2.26042829e-05, -9.89870768e-03, -7.14665146e-03, ...,
        -1.25975021e-02,  8.96067960e-06, -5.09378034e-03]])

In [70]:
npzfile['arr_2']

array(['Lifestyle', 'Lifestyle', 'Lifestyle', 'Lifestyle', 'Lifestyle',
       'Lifestyle', 'Lifestyle', 'Physical Activity', 'Lifestyle',
       'Lifestyle', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet',
       'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet',
       'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Diet', 'Alcohol',
       'Alcohol', 'Alcohol', 'Alcohol', 'Alcohol', 'Alcohol', 'Alcohol',
       'Alcohol', 'Alcohol', 'Tobacco', 'Tobacco', 'Tobacco', 'Tobacco',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Measurements',
       'Physical Measurements', 'Physical Meas

In [71]:
npzfile['arr_3']

array([  160,   182,   184,   186,   188,   418,   430,   704,   754,
         805,  1204,  1209,  1454,  1517,  1552,  1656,  1657,  1659,
        1817,  1866,  1942,  2024,  2027,  2059,  2075,  2225,  2301,
        2394,  2421,  2613,  2729,  2778,  2780,  2796,  2798,  2812,
        2844,  2865,  2870,  2873,  2922,  3019,  3021,  3036,  3048,
        3050,  3052,  3060,  3152,  3164,  3166,  3167,  3183,  3185,
        3187,  3189,  3191,  3218,  3220,  3222,  3224,  3230,  3231,
        3233,  3235,  3237,  3243,  3245,  3248,  3250,  3252,  3254,
        3259,  3261,  3264,  3270,  3272,  3274,  3278,  3281,  3285,
        3291,  3295,  3305,  3314,  3316,  3318,  3320,  3322,  3324,
        3330,  3331,  3332,  3334,  3336,  3338,  3340,  3347,  3354,
        3375,  3377,  3379,  3380,  3382,  3384,  3386,  3388,  3390,
        3392,  3396,  3398,  3400,  3401,  3402,  3404,  3406,  3408,
        3410,  3412,  3414,  3416,  3418,  3420,  3422,  3424,  3426,
        3428,  3430,

In [ ]:
# corr_df, bonferroni_threshold, false_discovery_rate = correlations_and_plots(
#                                     age_delta_decon = age_delta_decon, deconf_subset = vars_i_deconf_subset, 
#                                     names_subset = varsHeader_subset, idxs = varsIDX_subset,
#                                     categories_subset = vars_categories_subset,
#                                     title = modality, corr_flag='vars')

In [75]:
vars_categories_subset

array(['Ethnic Background', 'Ethnic Background', 'Ethnic Background', ...,
       'Mental Health', 'Mental Health', 'Mental Health'], dtype='<U35')

In [3]:
subjects_to_be_ignored = [21269692]
subject_sex = 'female'
prototype_flag = True
permutation_n_resamples = 276200 # otherwise 5000
load_top_flag = True

deconfound_flag = True
vars_cutoff_nan = 0.1

# Identify Correct File to Load

IDP_file = '../Analsysis Female New/female_test_IDPs.h5'
confounds_file = '../Analsysis Female New/female_test_conf12.h5'
data_file = 'female_test.pkl'

confounds = h5py.File(confounds_file, 'r')
confounds = confounds['conf12'][()]

IDP_h5 = h5py.File(IDP_file, 'r')
subjects_h5 = IDP_h5['subjects'][()]

# Load the relevant dataframe based on the required modality

df = pd.read_pickle(data_file)

IDP_h5 = h5py.File(IDP_file, 'r')
vars_i_deconf = IDP_h5['vars_i_deconf'][()]
subjects_h5 = IDP_h5['subjects'][()]

with open('../Analsysis Female New/varsHeader.txt') as f:
    varsHeader = f.readlines()
    varsHeader = [l.strip('\n\r') for l in varsHeader]
    varsHeader = np.array(varsHeader)
with open('../Analsysis Female New/vars_categories.txt') as f:
    vars_categories = f.readlines()
    vars_categories = [l.strip('\n\r') for l in vars_categories]
    vars_categories = np.array(vars_categories)

varsIDX = np.arange(0, len(varsHeader), 1).astype(int)

vars_perc_not_nan = np.sum(np.isnan(vars_i_deconf)==False,axis=0) / vars_i_deconf.shape[0]
vars_idx_nan_cutoff = np.where(vars_perc_not_nan >= vars_cutoff_nan)[0]

varsHeader_subset = varsHeader[vars_idx_nan_cutoff]
vars_i_deconf_subset = vars_i_deconf[:, vars_idx_nan_cutoff]
vars_categories_subset = vars_categories[vars_idx_nan_cutoff]
varsIDX_subset = varsIDX[vars_idx_nan_cutoff]

del IDP_h5

vars_corrs = []
vars_mods = []
vars_bnf_thrs = []
vars_fdr_thrs = []

In [4]:
def deconfound_inputs(y):
    
    y = y - y.mean(axis=0)
    if np.sum(np.isnan(y)) == 0:
        beta = np.linalg.pinv(confounds).dot(y)
        beta[np.abs(beta) < 1e-10] = 0
        yd = y - confounds.dot(beta)
        yd = yd - yd.mean()
    else:
        print('ERROR! Encoundered a NaN! Function Needs Updating!')
    
    return yd

def apply_FDR_correction(p_values):
    '''
    Code addapted from Emma Bluemke & Nicola Dinsdale
    Wrapper for Benjamini/Hochberg (non-negative) p-value correction for multiple tests.
    '''
    p_values_corrected = mt.multipletests(p_values, alpha=0.05, method='fdr_bh')[1] 
    
    return p_values_corrected


def standardize_data(variables):
    '''
    Code addapted from Emma Bluemke & Nicola Dinsdale
    '''

    number_of_subjects=variables.shape[0]

    # Compute the arithmetic mean & std along the specified axis, ignoring NaNs.
    variables_mean_ignore_NaNs = np.nanmean(variables,axis=0)
    variables_std_ignore_NaNs = np.nanstd(variables,axis=0)

    # We standardize the data
    variables_scaled = variables - np.tile(variables_mean_ignore_NaNs,(number_of_subjects,1))
    variables_scaled = variables_scaled / np.tile(variables_std_ignore_NaNs,(number_of_subjects,1))

    # Calculate how many variables are non NaN
    number_of_non_NaN =np.sum(np.isnan(variables)==False,axis=0) #np.nanstd has N**0.5 in divisor

    return variables_scaled, number_of_non_NaN


def align_subjects(subjects_df, subjects_h5, age_delta_decon, subjects_to_be_ignored):
    '''
    Here we make sure that the subjects match between the results and the nIDP/IDP dataframes
    The subjects_df uses slightly older data, and subjects might have left the study since it's generation
    If need be, this will be corrected at a later stage!
    '''
    
    if len(subjects_df) == len(subjects_h5):
        assert (subjects_df == subjects_h5).all()
    else:
        idx_elim = np.where(subjects_df == subjects_to_be_ignored)[0][0]
        subjects_df = np.delete(subjects_df, idx_elim)
        assert (subjects_df == subjects_h5).all()
        age_delta_decon = np.delete(age_delta_decon, idx_elim)
    return subjects_df, age_delta_decon


def statistic(x, y):
    return pearsonr(x,y)[0]
    

def correlate_with_fdr_correction(deltas, ukb_variables, permutation_n_resamples=5000):
    """
    Code addapted from Emma Bluemke & Nicola Dinsdale

    correlate age deltas with rows in ukb_variables
    returns pearson_r, t_test_statistic, p_values_corrected, p_values of length number_of_variables

    pearson_r = raw Pearson correlation value (between -1 and 1)
    t_test_statistic t test statistic for each variable ()
    p_values_corrected is corrected p-values values (fdr corrected)
    p_values is non corrected p-values
    """

    ukb_variables_scaled , ukb_variales_number_of_non_NaN = standardize_data(ukb_variables)
    ukb_variables_NaN_matrix = np.isnan(ukb_variables_scaled)
    ukb_variables_scaled[ukb_variables_NaN_matrix] = 0

    sys.stdout.flush()

    number_of_variables = ukb_variables_scaled.shape[1]

    deltas_array = np.tile(deltas,(number_of_variables,1)).T
    deltas_array[ukb_variables_NaN_matrix] = np.nan
    deltas_array = standardize_data(deltas_array)[0]
    deltas_array[ukb_variables_NaN_matrix] = 0

#     pearson_r = np.sum(deltas_array * ukb_variables_scaled, axis=0) / ukb_variales_number_of_non_NaN
    pearson_r = np.zeros(deltas_array.shape[1])
    p_values = np.zeros(deltas_array.shape[1])
    
    for idx in range(pearson_r.shape[0]):
        # CHANGE HERE THAT the zeros at nan-matrix locations are discarded, so we have only valid values for permutation testing
        deltas_array_permute = np.delete(deltas_array[:,idx], ukb_variables_NaN_matrix[:,idx])
        ukb_variables_scaled_permute = np.delete(ukb_variables_scaled[:,idx], ukb_variables_NaN_matrix[:,idx])
        
        res = permutation_test(
            (deltas_array_permute, ukb_variables_scaled_permute), 
            statistic, 
            permutation_type='pairings', 
            n_resamples=permutation_n_resamples, 
            random_state=1
        )
        
        pearson_r[idx] = res.statistic
        p_values[idx] = res.pvalue
        
        print("\r Processed {:.3f}%: {}/{} permutations".format(idx/len(pearson_r) * 100.0, idx+1, len(pearson_r)), end='')
    
    print('\n')
    
    t_test_statistic = pearson_r * ((ukb_variales_number_of_non_NaN-2)/(1-pearson_r**2))**0.5
    t_test_statistic = np.abs(t_test_statistic)
    t_test_statistic[ukb_variales_number_of_non_NaN <= 2] = np.nan #set those that had d.o.f 0 to nan

#     p_values=np.zeros(len(t_test_statistic))
    for j in range(len(t_test_statistic)):
        if t_test_statistic[j] is not np.nan:
#             p_values[j] = student_t.sf(t_test_statistic[j], ukb_variales_number_of_non_NaN[j] - 2) * 2
            pass
        else:
            p_values[j]=np.nan

    p_values_NaN_matrix = np.isnan(p_values)
    p_values_no_NaN = p_values[p_values_NaN_matrix==False]
    p_values_corrected_raw = apply_FDR_correction(p_values_no_NaN)

    p_values_corrected=np.zeros(len(p_values))
    p_values_corrected[p_values_NaN_matrix==False] = p_values_corrected_raw
    p_values_corrected[p_values_NaN_matrix==True] = np.nan

    return pearson_r, t_test_statistic, p_values_corrected, p_values


def correlations_and_plots(age_delta_decon, deconf_subset, names_subset, idxs, categories_subset, title, 
                           corr_flag='IDP', permutation_n_resamples=5000):
    
    corr_results = correlate_with_fdr_correction(age_delta_decon, deconf_subset, 
                                                 permutation_n_resamples=permutation_n_resamples)
    
    corr_df = pd.DataFrame.from_dict({
        'idx': idxs,
        "names": names_subset,
        'Categories': categories_subset,
        "pearson_r": corr_results[0],
        "t_test_statistic": corr_results[1],
        "p_values_corrected": corr_results[2],
        "p_values": corr_results[3],
        "abs_pearson_r": np.abs(corr_results[0]),
        "log_p_values": -np.log10(corr_results[3]),
        "log_p_values_corrected": -np.log10(corr_results[2]),
    })
    
    if corr_flag == 'IDP':
        short_p_values = np.extract(corr_df.p_values !=0, corr_df.p_values)
        bonferroni_threshold = 0.05/len(short_p_values) #... Bonferroni
    else:
        bonferroni_threshold = 0.05/len(corr_df.p_values) #... Bonferroni
    bonferroni_threshold = -np.log10(bonferroni_threshold)

    arguments = np.argsort(corr_df.p_values_corrected)
    sorted_p_values_corrected = corr_df.p_values_corrected[arguments]
    sorted_p_values = corr_df.p_values[arguments]
    if len(sorted_p_values_corrected[sorted_p_values_corrected<=0.05]):
        false_discovery_rate = np.nanmax(sorted_p_values[sorted_p_values_corrected<=0.05])
        false_discovery_rate = -np.log10(false_discovery_rate)
    else:

        false_discovery_rate = None
    
    return corr_df, bonferroni_threshold, false_discovery_rate

In [5]:
idx = 0

In [19]:
if load_top_flag == True:

    filename = 'logpOver2_' + str(idx) + '.npz'
    print(filename)

    npzfile = np.load(filename)
    varsHeader_subset = np.copy(npzfile['arr_0'])
    vars_i_deconf_subset = np.copy(npzfile['arr_1']).T
    vars_categories_subset = np.copy(npzfile['arr_2'])
    varsIDX_subset = np.copy(npzfile['arr_3'])
    del npzfile

df_mod = df.iloc[idx].dataframe
modality = df.iloc[idx].modality
age_delta_decon = df_mod.age_delta_decon.to_numpy()
subjects = df_mod['Unnamed: 0'].to_numpy()
subjects, age_delta_decon = align_subjects(subjects, subjects_h5, 
                                              age_delta_decon, subjects_to_be_ignored)

if deconfound_flag == True:
    age_delta_decon = deconfound_inputs(age_delta_decon)

logpOver2_0.npz


In [20]:
age_delta_decon = age_delta_decon
deconf_subset = vars_i_deconf_subset

In [21]:
deltas =  age_delta_decon
ukb_variables = deconf_subset
permutation_n_resamples=permutation_n_resamples

In [22]:
ukb_variables_scaled , ukb_variales_number_of_non_NaN = standardize_data(ukb_variables)
ukb_variables_NaN_matrix = np.isnan(ukb_variables_scaled)
ukb_variables_scaled[ukb_variables_NaN_matrix] = 0

sys.stdout.flush()

number_of_variables = ukb_variables_scaled.shape[1]

In [23]:
deltas_array = np.tile(deltas,(number_of_variables,1)).T
deltas_array[ukb_variables_NaN_matrix] = np.nan
deltas_array = standardize_data(deltas_array)[0]
deltas_array[ukb_variables_NaN_matrix] = 0

In [12]:
ukb_variables_NaN_matrix.shape

(345, 5021)

In [13]:
ukb_variables_scaled.shape

(345, 5021)

In [14]:
ukb_variales_number_of_non_NaN.shape

(5021,)

In [15]:
ukb_variables.shape

(345, 5021)

In [18]:
vars_i_deconf_subset.shape

(5021, 345)